In [6]:
import ipystata
import pandas as pd

### ipystata set up: 
###  replace stata location with your own & 
###  uncomment below 2 lines to set up ipystata in batch mode
#from ipystata.config import config_stata  
#config_stata('C:\Program Files\Stata16\StataIC-64', force_batch=True)

In [10]:
df = pd.read_stata('data/countymurders.dta')
df.head(4)

,arrests,countyid,density,popul,perc1019,perc2029,percblack,percmale,rpcincmaint,rpcpersinc,rpcunemins,year,murders,murdrate,arrestrate,statefips,countyfips,execs,lpopul,execrate
0,2.0,1001,54.049999,32216.0,20.629999,15.28,22.330000,40.250000,167.669998,8780.799805,29.160000,1980,2,0.620810,0.620809,1,1,0,10.380219,0.0
1,3.0,1001,53.660000,31984.0,20.190001,15.55,22.070000,40.360001,167.990005,8232.799805,43.919998,1981,1,0.312656,0.937969,1,1,0,10.372991,0.0
2,2.0,1001,53.750000,32036.0,19.660000,15.73,21.799999,40.419998,166.630005,8327.610352,71.410004,1982,3,0.936446,0.624298,1,1,0,10.374616,0.0
3,7.0,1001,53.779999,32056.0,19.100000,15.88,21.530001,40.470001,176.529999,8545.549805,72.220001,1983,7,2.183679,2.183679,1,1,0,10.375239,0.0


Send the dataframe to stata with the -d command 

Declare the it as panel data to stata with panel/time varibles

Use xtreg, fe to define a linear model using the fixed-effects estimator

In [17]:
%%stata -d df

xtset countyid year
xtreg arrests density popul perc1019 perc2029 percblack percmale rpcincmaint rpcpersinc rpcunemins murders murdrate arrestrate execs i.year, fe vce(robust)


       panel variable:  countyid (strongly balanced)
        time variable:  year, 1980 to 1996
                delta:  1 unit

Fixed-effects (within) regression               Number of obs     =     36,842
Group variable: countyid                        Number of groups  =      2,197

R-sq:                                           Obs per group:
     within  = 0.1787                                         min =          7
     between = 0.1198                                         avg =       16.8
     overall = 0.1020                                         max =         17

                                                F(29,2196)        =      51.50
corr(u_i, Xb)  = -0.8006                        Prob > F          =     0.0000

                           (Std. Err. adjusted for 2,197 clusters in countyid)
------------------------------------------------------------------------------
             |               Robust
     arrests |      Coef.   Std. Err.      t    P>|t|     

In [23]:
%%stata -d df

xtset countyid year
xtreg arrests density popul perc1019 perc2029 percblack percmale rpcincmaint rpcpersinc rpcunemins murders murdrate arrestrate execs i.year, fe vce(bootstrap)


       panel variable:  countyid (strongly balanced)
        time variable:  year, 1980 to 1996
                delta:  1 unit
(running xtreg on estimation sample)

Bootstrap replications (50)
----+--- 1 ---+--- 2 ---+--- 3 ---+--- 4 ---+--- 5 
..................................................    50

Fixed-effects (within) regression               Number of obs     =     36,842
Group variable: countyid                        Number of groups  =      2,197

R-sq:                                           Obs per group:
     within  = 0.1787                                         min =          7
     between = 0.1198                                         avg =       16.8
     overall = 0.1020                                         max =         17

                                                Wald chi2(29)     =    1411.61
corr(u_i, Xb)  = -0.8006                        Prob > chi2       =     0.0000

                            (Replications based on 2,197 clusters in countyid